In [2]:
from datasets import concatenate_datasets, load_dataset, load_from_disk

# bookcorpus = load_dataset("bookcorpus", split="train")
bookcorpus = load_from_disk("../processed_datadir/bookcorpus-raw-train")



In [3]:
for i in range(10):
    print(bookcorpus[i]["text"])
    print("--------")

usually , he would be tearing around the living room , playing with his toys .
--------
but just one look at a minion sent him practically catatonic .
--------
that had been megan 's plan when she got him dressed earlier .
--------
he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .
--------
she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .
--------
`` are n't you being a good boy ? ''
--------
she said .
--------
mason barely acknowledged her .
--------
instead , his baby blues remained focused on the television .
--------
since the movie was almost over , megan knew she better slip into the bedroom and finish getting ready .
--------


In [7]:
# bookcorpus.save_to_disk("../bookcorpus-raw-train")
# wiki = load_dataset("wikipedia", "20220301.en", split="train")
wiki = load_from_disk("../processed_datadir/wikipedia-raw-train")

In [ ]:
for i in range(10):
    print(wiki[i]["text"])
    print("--------")

In [5]:
# wiki.save_to_disk("../wikipedia-raw-train")

Saving the dataset (0/41 shards):   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-gpt2-2048/")

In [18]:
temp = tokenizer(wiki[0]['text'], return_special_tokens_mask=True)
for i in temp.keys():
    print(i)
    print(len(temp[i]))
temp = tokenizer(wiki[0]['text'],truncation=True, max_length=tokenizer.model_max_length)
for i in temp.keys():
    print(len(temp[i]))

input_ids
8287
attention_mask
8287
special_tokens_mask
8287
2048
2048


In [8]:
print(tokenizer.mask_token_id)

103


In [10]:

wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column

assert bookcorpus.features.type == wiki.features.type
raw_datasets = concatenate_datasets([bookcorpus, wiki])

In [19]:
from transformers import AutoTokenizer
import multiprocessing

# load tokenizer
# 跑下面这个代码可能需要几个小时，注意使用python

num_proc = multiprocessing.cpu_count()-4
print("num_proc",num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    tokenized_inputs = tokenizer(
       examples["text"], 
       #return_special_tokens_mask=True, 
       #truncation=True, max_length=tokenizer.model_max_length  # 这里不应该有这句话
    )
    print("success")
    return tokenized_inputs

# preprocess dataset
tokenized_datasets = raw_datasets.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features


num_proc 12
The max length for the tokenizer is: 2048


Map (num_proc=12):   0%|          | 0/80462898 [00:00<?, ? examples/s]

success
success
success
success
success
success
success
successsuccess

success
success
success
success
successsuccess

successsuccesssuccess


success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
successsuccess

success
success
success
success
success
success
success
successsuccess

successsuccess

success
successsuccess

success
success
success
success
success
success
success
success
success
success
success
successsuccess

success
success
successsuccess

success
success
successsuccess

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
successsuccesssuccesssuccess
successsuccess




successsuccess
success
success

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
successsuccess

success
success
successs

KeyboardInterrupt: 

In [ ]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= tokenizer.model_max_length:
        total_length = (total_length // tokenizer.model_max_length) * tokenizer.model_max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + tokenizer.model_max_length] for i in range(0, total_length, tokenizer.model_max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=num_proc)
# shuffle dataset
tokenized_datasets = tokenized_datasets.shuffle(seed=34)

print(f"the dataset contains in total {len(tokenized_datasets)*tokenizer.model_max_length} tokens")
# the dataset contains in total 3417216000 tokens
